# Read Eskom tarifs from their website

In [1]:
import requests
from html.parser import HTMLParser
import pathlib
import pandas as pd
import re
import sqlite3

In [2]:
url = 'https://www.eskom.co.za/distribution/rates-in-excel-format/'

In [3]:
rates = pathlib.Path('rates')
rates.mkdir(exist_ok=True)

In [4]:
class LocalParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        attrdict = dict(attrs)
        if tag == 'a' and 'xls' in (href := attrdict['href']):
            filename = href.split('/')[-1]
            target = rates / filename
            if not target.exists():
                print(f'Downloading {href}')
                target.write_bytes(requests.get(href).content)
            else:
                print(f'Already downloaded {target}')

In [5]:
LocalParser().feed(requests.get(url).text)

Already downloaded rates/Rates-in-excel-format.xlsm
Already downloaded rates/Eskom-Tariff-book-1-April-2021-V0.xlsm
Already downloaded rates/202021-Eskom-Tariffs.xlsm
Already downloaded rates/2019_20-Tariff-book-excel-version-1-April-2019-internet-Rev-00.xlsm
Already downloaded rates/Rates-in-Excel-format-2018_19.xlsm
Already downloaded rates/201718-Tariff-book-Scenario-internet-version.xlsm
Already downloaded rates/2016_17-Tariff-rates-published-version-v00.xlsm
Already downloaded rates/Tariff-book-2015_16-excl-Transflex.xlsm
Already downloaded rates/201415-Tariff-book-Internet-version.xlsm
Already downloaded rates/Tariffs-2013_14-Internet.xlsm


In [6]:
year_re = re.compile(r'\d{4}.\d{2}')

To save time, I'm assuming some stuff about these tarifs:
- There are 2 blocks
- The cutoff is 600 kWh
- The rate we're interested in is the +VAT one
- Rates apply from April in the first named year to March in the second

In [7]:
import datetime

In [8]:
rate_data = []
for f in rates.glob('*.xlsm'):
    ef = pd.ExcelFile(f)
    df = (
        ef.parse(sheet_name='Menu')
        .dropna(axis='columns', how='all')
        .dropna(how='all')
    )
    year = None
    for item in df.values.ravel():
        if not isinstance(item, str):
            continue
        for m in year_re.findall(item):
            year = m
            break
        if year:
            break
    print(year)
    sheet = [s for s in ef.sheet_names if 'Homepower' in s and 'Munic' not in s][0]
    print(sheet)
    df = ef.parse(sheet_name=sheet).dropna(how='all').dropna(axis='columns', how='all')
    keepcols = []
    startrow = None
    for c in df.columns:
        for row, v in enumerate(df[c]):
            if not isinstance(v, str):
                continue
            if 'Block' in v:
                startrow = row
            if 'Block' in v or 'VAT' in v:
                keepcols.append(c)
                break
    valueblock = df.iloc[startrow:startrow+2][keepcols].values
    rate_data.append({
        'Date': datetime.datetime(int(year.split('/')[0]), 4, 1),
        'Block 1': valueblock[1, 1],
        'Block 2': valueblock[1, 3],
    }
    )
elec = pd.DataFrame(rate_data).set_index('Date').sort_index()

2022/23
Homepower NLA
2021/22
Homepower NLA
2018/19
Homepower inc
2013/14
Homepower inc
2020/21
Homepower NLA
2015/16
Homepower inc
2017/18
Homepower inc
2019/20
Homepower NLA
2016/17
Homepower inc
2013/14
Homepower inc


In [9]:
con = sqlite3.connect('costs.db')

In [11]:
elec.to_sql('electricity', con, if_exists='replace')

10